# A Mid-Infrared Analysis of Accreeting Supermassive Black Holes 
## Utilzing Diagnostic Tools to Identify Active Galactic Nuclei (AGN)
### Part 2: Evaluating the Completness and Reliability of the AGN Selection Criteria
As any selection can suffer from incompleteness and contamination, there is a need to choose a selection that is both reliable, and complete. In this notebook, we will evaluate the completeness and reliability of the AGN selection criteria.

Given the galaxies that were selected as AGN candidates, we will compare the selection diagnostic against a truth sameple of known AGN. These known AGN have been detected by the Code Investigating GALaxy Emission (CIGALE) fitting code.

To determine the completeness and reliability of the selection criteria, we will use the following definitions:

$$\text{Completeness} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN in the truth sample}}$$

$$\text{Reliability} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN selected by the criteria}}$$

In [71]:
# Begin by importing the required packages for the project
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import os

#import seaborn as sns

In [72]:
# define an import function for data handling
def import_AGN_data(field, all_data, selection_criteria, e_sigma, folder_path = 'diagnostic selections'):
    
    if all_data == True:
        file_name =  field+'_full_data_'+str(e_sigma)+'_sigma.csv'
        file_path = os.path.join(folder_path, file_name)
        selection_df = pd.read_csv(file_path)
    else:
        if selection_criteria == 'Lacy':
            file_name = field+'_lacy_selection_'+str(e_sigma)+'_sigma.csv'
            file_path = os.path.join(folder_path, file_name)
            selection_df = pd.read_csv(file_path)
        elif selection_criteria == 'Messias':
            file_name = field+'_messias_selection_'+str(e_sigma)+'_sigma.csv'
            file_path = os.path.join(folder_path, file_name)
            selection_df = pd.read_csv(file_path)
        elif selection_criteria == 'Donley':
            file_name = field+'_donley_selection_'+str(e_sigma)+'_sigma.csv'
            file_path = os.path.join(folder_path, file_name)
            selection_df = pd.read_csv(file_path)
        elif selection_criteria == 'Combined':
            file_name = field+'_combined_selection_'+str(e_sigma)+'_sigma.csv'
            file_path = os.path.join(folder_path, file_name)
            selection_df = pd.read_csv(file_path)
        else:
            print('Please enter a valid selection criteria')
            return 0
    return selection_df

In [73]:
# We now need to read in the dataframes that were created in the previous notebook for all fields
# We will also need to read in the truth samples that were created from the CIGALE code

# Choose a field to work with
field = 'CDFS'

# Choose the error threshold for comparison
sigma = 5

# import relevant AGN data.
selections_df = import_AGN_data(field, False,'Combined', sigma, 'diagnostic selections') 



In [74]:

# Note, FITS files by virtue of their structure are big-endian, so we need to swap the bytes to use them in 
# Pandas Dataframes, as these are little-endian by default.

# Read in the truth samples and then create dataframes from the fits files
truth_sample = fits.open(field+'_truth.fits')
truth_df=pd.DataFrame(np.array(truth_sample[1].data).byteswap().newbyteorder()) # Byteswap so that Pandas can read it
truth_df.rename(columns={'id_1':'id'}, inplace=True) # Rename the ID column so that it matches the other dataframes

In [75]:
# Check the truth_df dataframe
truth_df.head()

,id,bayes.agn.total_dust_luminosity,bayes.agn.total_dust_luminosity_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_star,bayes.stellar.m_star_err
0,5746,2.834282e+38,6.358570e+37,5.556047e+38,2.778023e+37,177.095988,8.854799,3.368743e+10,1.695740e+09
1,5860,3.109523e+37,4.202669e+37,7.505922e+38,5.678178e+37,203.540655,21.019983,4.616899e+10,8.537885e+09
2,5880,1.582654e+36,3.727870e+36,6.653951e+37,3.326975e+36,20.495446,1.024772,1.296424e+10,1.792094e+09
3,6020,1.065299e+38,1.218670e+37,2.165540e+38,2.411809e+37,52.781956,5.878316,2.554253e+10,2.845202e+09
4,6153,1.612261e+38,1.584376e+37,4.013813e+38,2.006907e+37,97.940061,4.897003,4.713909e+10,2.356954e+09


In [76]:
# bayes.agn.total_dust_luminosity <- AGN contribution
truth_df['bayes.agn.total_dust_luminosity']

# bayes.dust.luminosity <- stellar contribution
truth_df['bayes.dust.luminosity']

# filter the truth data based on error
truth_df = truth_df[truth_df['bayes.agn.total_dust_luminosity'] > 0]
truth_df = truth_df[truth_df['bayes.dust.luminosity'] > 0]



CIGALE uses a Bayesian approach with MCMC techniques. CIGALE allows us to analyse SED data, separatating the contributions of the AGN from the stellar-heated dust.  In the truth sample the luminosity from these two dust components can be used to derive a correct truth sample. A truth sample of AGN will be found if the luminosity contribution from the AGN is greater than 50% of the total luminosity of the system (AGN + Dust).

In [77]:
# To find the AGN we look at the values of the AGN's luminosity, and compare it against the
# total luminosity of that galaxy. If the lumniosty of the AGN is greater than 50% of the total
# luminosity of the entire galaxy, we have an AGN.


# This will be the AGN luminosity contribution
truth_df['agn contribution'] = truth_df['bayes.agn.total_dust_luminosity']/(truth_df['bayes.agn.total_dust_luminosity'] + truth_df['bayes.dust.luminosity'])
truth_df

,id,bayes.agn.total_dust_luminosity,bayes.agn.total_dust_luminosity_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_star,bayes.stellar.m_star_err,agn contribution
0,5746,2.834282e+38,6.358570e+37,5.556047e+38,2.778023e+37,177.095988,8.854799,3.368743e+10,1.695740e+09,0.337803
1,5860,3.109523e+37,4.202669e+37,7.505922e+38,5.678178e+37,203.540655,21.019983,4.616899e+10,8.537885e+09,0.039780
2,5880,1.582654e+36,3.727870e+36,6.653951e+37,3.326975e+36,20.495446,1.024772,1.296424e+10,1.792094e+09,0.023233
3,6020,1.065299e+38,1.218670e+37,2.165540e+38,2.411809e+37,52.781956,5.878316,2.554253e+10,2.845202e+09,0.329728
4,6153,1.612261e+38,1.584376e+37,4.013813e+38,2.006907e+37,97.940061,4.897003,4.713909e+10,2.356954e+09,0.286569
...,...,...,...,...,...,...,...,...,...,...
881,30561,9.809550e+36,2.533053e+36,1.175783e+37,1.211326e+36,7.495037,0.393237,2.390450e+10,1.195225e+09,0.454833
882,30647,1.536118e+37,9.036213e+36,3.623751e+38,7.678264e+37,123.907630,30.253412,1.518658e+10,1.851144e+09,0.040666
883,30724,2.329137e+37,1.307431e+36,1.536297e+36,1.114254e+36,1.393609,0.276501,2.125778e+08,3.290448e+08,0.938122
884,30734,1.784442e+36,2.740955e+36,6.319084e+37,4.653180e+37,21.438396,12.942402,9.670290e+09,4.472081e+09,0.027463


In [78]:

# Add a new column for known AGN
truth_df['Known AGN'] = np.where(truth_df['agn contribution'] > 0.5, 1, 0)

# This will be the AGN luminosity contribution
num_true_AGN = len(truth_df[truth_df['Known AGN'] == 1])

print("There are " + str(num_true_AGN) + " AGN that have been found by CIGALE in the " + field + " field.")

There are 106 AGN that have been found by CIGALE in the CDFS field.


Now that we know the amount of AGN, we need to use the ID from the truth sample, and append this to the AGN candidates. This will allow us to compare the selection criteria against the truth sample.If the ID from the truth sample is found in the AGN candidates, then we know that the AGN candidate is also an AGN in the truth sample and thus a positive diagnostic selection

As we have two selection criteria (Lacy, Messias) we will have two seperate positive diagnostic columns for each of the criteria.



In [79]:
# join the two dataframes
selections_df = selections_df.join(truth_df.set_index('id'), on='id')

In [80]:
# Create a new column for positive diagnostic, this will be set to zero initally
selections_df['Positive Lacy Selection'] = 0
selections_df['Positice Messias Selection'] = 0
selections_df['Positive Donley Selection'] = 0

# We now need to test against the AGN to see if we have a positive selection


# First for Lacy, then Messias
selections_df['Positive Lacy Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Lacy Selection'] == 1), 1, 0)
selections_df['Positive Donley Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Donley Selection'] == 1), 1, 0)
selections_df['Positive Messias Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Messias Selection'] == 1), 1, 0)

In [81]:
# Now we can do a comparison to determine the reliability and completeness of the selections
selections_df['Positive Lacy Selection'].value_counts()[1]
selections_df['Positive Donley Selection'].value_counts()[1]
selections_df['Positive Messias Selection'].value_counts()[1]


25

We now can calculate the completeness of the of the selection, we do this using the criteria below

$$\text{Completeness} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN in the truth sample}}$$

In [82]:
def calculateCompleteness(df, diagnostic):
    # Calculate the completeness of the selection
    # Completeness = Positive Diagnostic / Known AGN
    # Positive Diagnostic = AGN that are selected by CIGALE, and as AGN by the selection diagnostic
    # Known AGN = AGN selected by CIGALE
    positive_selection = df['Positive '+ diagnostic + " Selection"].value_counts()[1]
    known_AGN = df['Known AGN'].value_counts()[1]
    return positive_selection/known_AGN

# Calculate the completeness of the Lacy and Messias selection
lacy_completeness = calculateCompleteness(selections_df, 'Lacy')
messias_completeness = calculateCompleteness(selections_df, 'Messias')
donley_completeness = calculateCompleteness(selections_df, 'Donley')

We now can calculate the reliability of the of the selection, we do this using the criteria below

$$\text{Reliability} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN selected by the criteria}}$$


In [83]:
def calculateReliability(df, diagnostic):
    # Calculate the reliability of the selection
    # Reliability = Positive Diagnostic / Positive Selection
    # Positive Diagnostic = AGN that are selected by CIGALE, and as AGN by the selection diagnostic
    # Positive Selection = AGN that are selected by the selection diagnostic
    positive_selection = df['Positive '+ diagnostic + " Selection"].value_counts()[1]
    diagnostic_selection = df[diagnostic + ' Selection'].value_counts()[1]
    return positive_selection/diagnostic_selection

# Calculate the reliability of the Lacy and Messias selection
lacy_reliability = calculateReliability(selections_df, 'Lacy')
messias_reliability = calculateReliability(selections_df, 'Messias')
donley_reliability = calculateReliability(selections_df, 'Donley')


selections_df[selections_df['Known AGN'] == 1]


,id,x,y,ra,dec,SEflags,iso_area,fap_Ksall,eap_Ksall,apcorr,...,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_star,bayes.stellar.m_star_err,agn contribution,Known AGN,Positive Lacy Selection,Positice Messias Selection,Positive Donley Selection,Positive Messias Selection
7,6465,4590.306,2328.860,53.076000,-27.878149,0,67.0,2.250058,0.082577,1.152161,...,14.572617,2.389311,5.606788e+09,1.068463e+09,0.780016,1.0,1,0,1,0
20,6964,4519.159,2508.198,53.079357,-27.870678,3,354.0,1.968985,0.069741,1.018602,...,25.279656,1.828421,4.025984e+09,1.566494e+09,0.574379,1.0,1,0,1,1
25,7267,4369.300,2512.335,53.086422,-27.870508,2,193.0,4.899882,0.069986,1.077356,...,0.240667,0.023940,5.424387e+08,5.537860e+07,0.590943,1.0,1,0,0,0
37,7814,4478.820,2607.099,53.081261,-27.866558,0,86.0,1.336565,0.068553,1.114034,...,14.699932,3.150302,5.411311e+09,1.795156e+09,0.579789,1.0,1,0,1,0
45,8174,4375.527,2684.442,53.086128,-27.863337,0,105.0,2.327794,0.066987,1.108162,...,10.813307,2.059831,1.368593e+10,8.171281e+08,0.591034,1.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,29900,4347.313,7207.462,53.087521,-27.674877,3,215.0,9.582683,0.170237,1.083959,...,0.998413,0.403330,1.493157e+10,9.800267e+08,0.888589,1.0,1,0,0,0
1187,30290,2890.258,7403.900,53.156071,-27.666693,3,346.0,19.522750,0.137106,1.071010,...,0.829633,0.289808,1.424578e+10,1.613516e+09,0.863480,1.0,1,0,1,0
1189,30321,3126.935,7395.678,53.144936,-27.667038,3,145.0,4.184390,0.160500,1.088097,...,0.664756,0.134309,2.047170e+09,3.002494e+08,0.671159,1.0,1,0,0,0
1199,30724,3584.657,7482.463,53.123402,-27.663424,0,25.0,0.674413,0.181990,1.377135,...,1.393609,0.276501,2.125778e+08,3.290448e+08,0.938122,1.0,1,0,1,1


## Conclusion
Bringing this all together we can see both the completeness and relaibility of each of the diagnostics. Below we see the outputs of the completeness and reliability tests for our Lacy and Messias diagnostics. 

In [84]:
print("The completeness of the Lacy selection is " + str(round(lacy_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Lacy selection is " + str(round(lacy_reliability*100, 2))+"% in the " + field + " field.")
print("\n")
print("The completeness of the Donley selection is " + str(round(donley_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Donley selection is " + str(round(donley_reliability*100, 2))+"% in the " + field + " field.")
print("\n")
print("The completeness of the Messias selection is " + str(round(messias_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Messias selection is " + str(round(messias_reliability*100, 2))+"% in the " + field + " field.")



The completeness of the Lacy selection is 97.17% in the CDFS field.
The reliability of the Lacy selection is 8.73% in the CDFS field.


The completeness of the Donley selection is 44.34% in the CDFS field.
The reliability of the Donley selection is 14.46% in the CDFS field.


The completeness of the Messias selection is 23.58% in the CDFS field.
The reliability of the Messias selection is 42.37% in the CDFS field.


# Export Reliability and Completeness Table 
Export the table to a csv file for further analysis.

The python script Selection Criteria Comparison will do this.
